In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor


testdata = './Equipment data from keggel/test.csv'
traindata = './Equipment data from keggel/train.csv'


test = pd.read_csv(testdata)
train = pd.read_csv(traindata)
display(train)
train.isnull().sum()
train.dropna(inplace=True)
# Remove the last 5 rows from the test data
train = train.drop(['Product ID'], axis=1)
display(train)
# Change the type column to numerical values
train['Type'] = train['Type'].replace({'L': 1, 'M': 2, 'H': 3})
display(train)
plt.figure(figsize=(20,10))
sns.heatmap(train.corr(),annot=True, cmap='coolwarm')
plt.title ('Correlation Heatmap')
plt.show()
# Feature engneering 
train['Temprature_Diffrence'] = train['Process temperature [K]'] - train['Air temperature [K]']
display(train)
# Code that lists where machine failures are happening
machine_failure = train[train['Machine failure'] == 1]

display(machine_failure)
failure_counts = train['Machine failure'].value_counts()
plt.figure(figsize=(6,4))
sns.barplot(x=failure_counts.index, y=failure_counts.values)
plt.title('Machine failure Count')
plt.xlabel('Machine failure')
plt.ylabel
plt.show()
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming `df` is your DataFrame
columns = ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']

for i in range(len(columns)):
    for j in range(i+1, len(columns)):
        plt.figure(figsize=(30, 17))
        sns.scatterplot(data=train, x=columns[i], y=columns[j], hue='Machine failure')
        plt.title(f'{columns[j]} vs {columns[i]}')
        plt.show()
    
import seaborn as sns

# Assuming `df` is your DataFrame
columns = ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']
sns.pairplot(train[columns])
# Violin plot of all the features vs machine failure
plt.figure(figsize=(30, 20))
for i, col in enumerate(train.columns[1:-1]):
    plt.figure(figsize=(10, 6))
    sns.violinplot(x='Machine failure', y=col, data=train)
    plt.title(f'Violin plot of {col} vs Machine failure')
    plt.xlabel('Machine failure')
    plt.ylabel(col)
    plt.show()

print(train.columns)
plt.figure(figsize=(12, 10))
sns.pairplot(train[['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]', 'Machine failure']], hue='Machine failure', diag_kind='kde', plot_kws={'alpha': 0.6})
plt.show()
# Calculating the Standard Deviation of the features
std = train.std()
print(std)

#  Create a model to predict the machine failure using the train data by support vector regression 
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X = train.drop(['Machine failure'], axis=1)
y = train['Machine failure']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svr = SVR()
svr.fit(X_train, y_train)
y_pred = svr.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse: .2f}')
# Calculate the accuracy of the model
from sklearn.metrics import accuracy_score
y_pred = np.round(y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy: .2f}')
# save the model to a file
import pickle 
filename = "Predictive_maintenance_model V2.pkl"
with open(filename, 'wb') as file:
    pickle.dump(svr, file)

# Third iteration on making a better model
# Making the model using Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

X = train.drop(['Machine failure'], axis=1)
y = train['Machine failure']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy: .2f}')
# Making a Model that not only classifies machine failure but also failure type

# Test the model using the test data loaded in the beginning
test = test.drop(['Product ID'], axis=1)
test['Type'] = test['Type'].replace({'L': 1, 'M': 2, 'H': 3})
test['Temprature_Diffrence'] = test['Process temperature [K]'] - test['Air temperature [K]']


y_pred = rf.predict(test)
display(y_pred)
# Saving the model to a file 
import pickle as pkl
filename = "Predictive_maintenance_model V3.pkl"
with open(filename, 'wb') as file:
    pkl.dump(rf, file)
